# EWG Tap Water Database Crawler

In [1]:

import os
import time
import csv
import asyncio
import json
from urllib.request import urlopen
import sys
import asyncio
import aiohttp
import json
from pprint import pprint
import requests
from pyquery import PyQuery as pq
# from sein import log
from itertools import compress
from collections import OrderedDict

def _start_or_end_with(text, pattern):
  pattern = tuple(pattern)
  return text.startswith(pattern) or text.endswith(pattern)



def save_csv(headers, content, path):
  with open(path, 'w', encoding='utf-8') as f:
    f_csv = csv.writer(f, lineterminator='\n')
    f_csv.writerow(headers)
    f_csv.writerows(content)
  print('save csv done', path)
  

def load_csv(path, sample=10, only_title=False, include=(), exclude=()):
  with open(path) as f:
    titles = f.readline().strip().split(',')

  if include:
    column_compress = [_start_or_end_with(title, include) for title in titles]
  else:
    column_compress = [True] * len(titles)
  if exclude:
    column_compress = [not _start_or_end_with(title, exclude) and tb for title, tb in zip(titles, column_compress)]

  if only_title:
    return list(compress(titles, column_compress))

  with open(path) as f:
    lines = csv.reader(f)
    next(lines)
    result = []
    for i, line in enumerate(lines, 1):
      if sample and i > sample:
        break
      # result.append([_load_csv_value_convert(x) for x in compress(line, column_compress)])
      result.append([x for x in compress(line, column_compress)])

  return result


def windows(iterable, length=2, overlap=0, yield_tail=False):
  '''按照固定窗口大小切片list, 可以重叠
  滑动array窗口,
  每次提供length数目的元素,如果有overlap则重复之前的元素
  yield_tail: 最后不足 length 的那部分元素是否也要 yield'''
  import itertools
  if length <= overlap:
    raise AttributeError('overlap {} cannot larger than length {}'.format(overlap, length))
  it = iter(iterable)
  results = list(itertools.islice(it, length))
  while len(results) == length:
    yield results
    results = results[length-overlap:]
    results.extend(itertools.islice(it, length-overlap))
  if results and yield_tail:
    yield results
    
def make_code_list(path):
    ret = []
    with open(path, 'r') as f:
        for line in f.readlines():
            if line.startswith('PWS ID') or line.startswith('pwsid') or line.strip() == '':
                continue
            else:
                ret.append(line.split(',')[0])
    return ret

In [ ]:

  
  
def merge_all_pwsids(output_file):
  filenames = '''epa_water_system_summary_01_[8].csv
  epa_water_system_summary_02_[19].csv
  epa_water_system_summary_03_[None].csv
  epa_water_system_summary_04_[120].csv
  epa_water_system_summary_05_[282].csv
  epa_water_system_summary_06_[151].csv
  epa_water_system_summary_07_[21].csv
  epa_water_system_summary_08_[239].csv
  epa_water_system_summary_09_[567].csv
  epa_water_system_summary_10_[202].csv
  epa_water_system_summary_AK_[3549].csv
  epa_water_system_summary_AL_[1810].csv
  epa_water_system_summary_AR_[4194].csv
  epa_water_system_summary_AS_[273].csv
  epa_water_system_summary_AZ_[3967].csv
  epa_water_system_summary_CA_[18131].csv
  epa_water_system_summary_CO_[4588].csv
  epa_water_system_summary_CT_[11043].csv
  epa_water_system_summary_DE_[1300].csv
  epa_water_system_summary_FL_[19678].csv
  epa_water_system_summary_GA_[5469].csv
  epa_water_system_summary_GU_[18].csv
  epa_water_system_summary_HI_[228].csv
  epa_water_system_summary_IA_[5201].csv
  epa_water_system_summary_ID_[4321].csv
  epa_water_system_summary_IL_[26598].csv
  epa_water_system_summary_IN_[9488].csv
  epa_water_system_summary_KS_[1683].csv
  epa_water_system_summary_KY_[2189].csv
  epa_water_system_summary_LA_[5051].csv
  epa_water_system_summary_MA_[3476].csv
  epa_water_system_summary_MD_[7800].csv
  epa_water_system_summary_ME_[6191].csv
  epa_water_system_summary_MI_[25680].csv
  epa_water_system_summary_MN_[20946].csv
  epa_water_system_summary_MO_[5698].csv
  epa_water_system_summary_MP_[203].csv
  epa_water_system_summary_MS_[3172].csv
  epa_water_system_summary_MT_[4581].csv
  epa_water_system_summary_NC_[23139].csv
  epa_water_system_summary_ND_[1591].csv
  epa_water_system_summary_NE_[2805].csv
  epa_water_system_summary_NH_[4003].csv
  epa_water_system_summary_NJ_[14658].csv
  epa_water_system_summary_NM_[2591].csv
  epa_water_system_summary_NN_[250].csv
  epa_water_system_summary_NV_[1364].csv
  epa_water_system_summary_NY_[25918].csv
  epa_water_system_summary_OH_[15903].csv
  epa_water_system_summary_OK_[5603].csv
  epa_water_system_summary_OR_[5802].csv
  epa_water_system_summary_PA_[22436].csv
  epa_water_system_summary_PR_[965].csv
  epa_water_system_summary_RI_[1112].csv
  epa_water_system_summary_SC_[5894].csv
  epa_water_system_summary_SD_[1538].csv
  epa_water_system_summary_TN_[4939].csv
  epa_water_system_summary_TX_[15282].csv
  epa_water_system_summary_UT_[2114].csv
  epa_water_system_summary_VA_[8721].csv
  epa_water_system_summary_VI_[986].csv
  epa_water_system_summary_VT_[4286].csv
  epa_water_system_summary_WA_[9465].csv
  epa_water_system_summary_WI_[23678].csv
  epa_water_system_summary_WV_[4337].csv
  epa_water_system_summary_WY_[1859].csv'''
  
  with open(output_file, 'w', encoding='utf-8') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(['pwsid', 'state', 'national', 'this_utility'])
    for filename in filenames.splitlines():
    # for filename in ['epa_water_system_summary_WI_[23678].csv']:
      codes = make_code_list('./epa_water_system/' + filename.strip())
      f_csv.writerows([(code, '', '', '') for code in codes])
      
# 收集所有的 pwsid 到指定文件
# merge_all_pwsids('ewg_all_sample.csv')

In [2]:
class NoDataException(Exception):
    pass

  
def fetch_page(url):
    r = requests.get(url)
    return r.content


def parse_nitrate_content(content):
    doc = pq(content)
    for div in doc.find('div.slide-toggle'):
      h3 = pq(div).find('p')
      if h3.text().startswith('Nitrate, a fertilizer chemical,'):
        nitrate_div = div
        break
    else:
      raise NoDataException('Not found Nitrate block')

    figure = pq(nitrate_div).find('figure.levels-compare-figure')

    state = figure.find('.state-ppb-popup').text()
    national = figure.find('.national-ppb-popup').text()
    utility = figure.find('.this-utility-ppb-popup').text()

    return state, national, utility
  

In [5]:


async def fetch_ewg_page(session, code):
    url_fmt = 'https://www.ewg.org/tapwater/system.php?pws={}'.format
    url = url_fmt(code)
    # print('async fetch_ewg_page {url}'.format(**locals()))
    async with session.get(url) as resp:
        body = await resp.read()
        try:
            data = parse_nitrate_content(body)
            ret = [code] + list(data)
            # print('done {url} {ret}'.format(**locals()))
        except NoDataException:
            ret = [code, 'nodata', 'nodata', 'nodata']
            # print('fail {url} data not_found'.format(**locals()))
        except UnicodeDecodeError:
            ret = [code, '', '', '']
            print('fail {url} unicode error'.format(**locals()))
#         with open(output_file, 'a') as output:
#             output.write('{},{},{},{},{}\n'.format(code, ret[0], ret[1], ret[2], url))
    return ret
    

  

def update_work_file(work_file, content, batch_results, count, title):
    for pwsid, state, national, utility in batch_results:
        content[pwsid] = state, national, utility
        if state == '':
            pass  # leave for next loop fetch
        elif state == 'nodata':
            count['nodata'] += 1
        else:
            count['done'] += 1
        count['task'] -= 1
      
    print('current count: task {task}, done {done}, nodata {nodata}'.format(**count))
    save_csv(title, [[k, *v] for k, v in content.items()], work_file)
    return 
  
  
def run_async_tasks(work_file, task_per_turn=10, sleep_interval=3):
  
    codes = []
    count = {'task': 0, 'done': 0, 'nodata': 0}
    
    title = load_csv(work_file, only_title=True)
    content = OrderedDict()

    for line in load_csv(work_file, sample=None):
        pwsid, state, national, utility = line
        content[pwsid] = state, national, utility
        
        if state == '':
            count['task'] += 1
            codes.append(pwsid)
        elif state == 'nodata':
            count['nodata'] += 1
        else:
            count['done'] += 1
            
            
    total_count = count['task'] + count['nodata'] + count['done']
    print('read work_file {work_file}, total {total_count}'.format(**locals()))
    print('current count: task {task}, done {done}, nodata {nodata}'.format(**count))
    # return
    # print(content)
    
    
    session = aiohttp.ClientSession()
    loop = asyncio.get_event_loop()
    
    batch_results = []
    for part in windows(codes, length=task_per_turn, yield_tail=True):
        print('send new tasks: {} ...  ({} items)'.format(' '.join(part[:5]), len(part)))
        tasks = [fetch_ewg_page(session, code) for code in part]
        turn_results = loop.run_until_complete(asyncio.gather(*tasks))
        # print('gather ', results)
        batch_results.extend(turn_results)
        
        if len(batch_results) >= 1000:
            update_work_file(work_file, content, batch_results, count, title)
            batch_results = []
        time.sleep(sleep_interval)
        
    if len(batch_results) >= 1:  # 最后残余的 code part
        update_work_file(work_file, content, batch_results, count, title)
        batch_results = []
        
run_async_tasks(work_file='ewg_all.csv', 
                task_per_turn=50,
                sleep_interval=1)


Creating a client session outside of coroutine
client_session: <aiohttp.client.ClientSession object at 0x10d49d438>


read work_file ewg_all.csv, total 419374
current count: task 48, done 22450, nodata 396876
send new tasks: CO0121100 CO0121150 CO0126188 CO0123010 CO0123538 ...  (48 items)
fail https://www.ewg.org/tapwater/system.php?pws=CO0126188 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=NC0363010 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=MT0000239 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=VT0020080 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=IN5253002 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=FL3054057 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=FL3481571 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=LA1069013 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=MD0100015 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=IN5253008 unicode error
fail https://www.ewg.org/tapwater/system.php?pws=CO0119786 unicode error
fail https://www.ewg.org

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10d49d438>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1190c3e10>, 9621.331255632), (<aiohttp.client_proto.ResponseHandler object at 0x11f1fc6d8>, 9621.358672039), (<aiohttp.client_proto.ResponseHandler object at 0x1190c3ba8>, 9621.391689982), (<aiohttp.client_proto.ResponseHandler object at 0x119054ba8>, 9621.420613943), (<aiohttp.client_proto.ResponseHandler object at 0x119062d30>, 9621.451672727), (<aiohttp.client_proto.ResponseHandler object at 0x1190c3080>, 9621.481768541), (<aiohttp.client_proto.ResponseHandler object at 0x11909fd68>, 9621.499209288), (<aiohttp.client_proto.ResponseHandler object at 0x11915db38>, 9621.521062096), (<aiohttp.client_proto.ResponseHandler object at 0x1190c3358>, 9621.553006079), (<aiohttp.client_proto.ResponseHandler object at 0x11900f240>, 9621.594410436), (<aiohttp.client_proto.ResponseHandler object at 0x11f5e9a90>, 9621.6233

save csv done ewg_all.csv
